In [1]:
import numpy as np
import itertools

# part 2 only shown below

In [2]:
main_program = '3,8,1001,8,10,8,105,1,0,0,21,46,59,84,93,110,191,272,353,434,99999,3,9,101,2,9,9,102,3,9,9,1001,9,5,9,102,4,9,9,1001,9,4,9,4,9,99,3,9,101,3,9,9,102,5,9,9,4,9,99,3,9,1001,9,4,9,1002,9,2,9,101,2,9,9,102,2,9,9,1001,9,3,9,4,9,99,3,9,1002,9,2,9,4,9,99,3,9,102,2,9,9,1001,9,5,9,1002,9,3,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,99'

In [3]:
import logging
from pathlib import Path
import os
# create logger with 'spam_application'
logger = logging.getLogger('spam_application')
logger.setLevel(logging.DEBUG)

# delete log if already exists
log_path = Path('spam.log')
if log_path.exists():
    os.remove(log_path)

# create file handler which logs even debug messages
fh = logging.FileHandler(log_path)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

# day07

In [4]:
op_code_num_params = {  1: 3, # add next two numbers, store according to 3rd param
                        2: 3, # multiply next two numbers, store according to 3rd param
                        3: 1, # takes integer and stores it at address given by only parameter (1 param)
                        4: 1, # outputs the value at the address of its only parameter (1 param)
                        5: 2, # jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter.
                        6: 2, # jump-if-false:
                        7: 3, # less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                        8: 3, # equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0.
                        99: 0} # ends program immediately (0 params)

In [5]:
param_modes = {0, # position mode - look at memory location
               1} # immediate mode - take value as written

In [6]:
def get_param_values(param_modes, params, program):
    param_values = []
    for mode, param in zip(param_modes, params):
        if mode == '0':
            param_values.append(program[param])
        elif mode == '1':
            param_values.append(param)
    return param_values

In [7]:
import copy

In [8]:
class amplifier(object):
    def __init__(self, name, program, input_queue=None, output_queue=None):
        self.name = name
        self.program = copy.copy(program)
        self.input_queue = input_queue
        self.output_queue = output_queue
        self.instr_ptr = 0
        self.stopped = False
        
    def run_until_inputop_and_empty_input_queue_or_end_of_program(self):
        continue_ = True
        next_opcode, _ = self.get_next_opcode_and_instr_modes()
        while not (next_opcode == 3 and len(self.input_queue) == 0):
            op_code = self.run_single_instruction()
            if op_code == 99:
                return
            next_opcode, _ = self.get_next_opcode_and_instr_modes()
        return
            
    
    def get_next_opcode_and_instr_modes(self):
        mode_and_op_code = int(self.program[self.instr_ptr])
    
        logger.info('reading op_code')
        op_code = mode_and_op_code % 100
        logger.info(f'op_code={op_code}')
        
        logger.info('getting param_modes')
        param_modes = str(mode_and_op_code // 100).zfill(op_code_num_params[op_code])[::-1] # left to right now
        logger.info(f'exited param_modes, param_modes = {param_modes}')
        return op_code, param_modes

        
    def run_single_instruction(self):
        logger.info(f'self.name:{self.name}')
        logger.info(f'program-state:{self.program}')
        logger.info(f'self.instr_ptr:{self.instr_ptr}')
        op_code, param_modes = self.get_next_opcode_and_instr_modes()
        
        logger.info('getting params')
        n_params = op_code_num_params[op_code]
        params = self.program[self.instr_ptr+1:self.instr_ptr+1+n_params]
        self.instr_ptr += n_params+1
        logger.info(f'done getting params, params={params}')

        logger.info('entering get_param_values')
        param_values = get_param_values(param_modes, params, self.program)
        logger.info(f'exited get_param_values, param_values={param_values}')

        # param_values are not necessarilly accurate except for op_code 1 and 2,
        # the logic was adjusted below to correct for this
        if op_code == 1:
            self.program[params[2]] = param_values[0] + param_values[1]
        elif op_code == 2:
            self.program[params[2]] = param_values[0] * param_values[1]
        elif op_code == 3:
            self.program[params[0]] = self.input_queue.pop(0)
        elif op_code == 4:
            self.output_queue.append(self.program[params[0]])
        elif op_code == 5:
            if param_values[0] != 0:
                self.instr_ptr = param_values[1]
        elif op_code == 6:
            if param_values[0] == 0:
                self.instr_ptr = param_values[1]
        elif op_code == 7:
            if param_values[0] < param_values[1]:
                self.program[params[2]] = 1
            else:
                self.program[params[2]] = 0
        elif op_code == 8:
            if param_values[0] == param_values[1]:
                self.program[params[2]] = 1
            else:
                self.program[params[2]] = 0
        elif op_code == 99:
            self.stopped = True
            
        logger.info(f'post_op program-state:{self.program}')
        logger.info(f'self.instr_ptr:{self.instr_ptr}')
        logger.info('\n')
        return op_code

In [9]:
test1 = '3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5'
test2 = '3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10'
program = main_program
program = [int(i) for i in program.split(',')]

In [12]:
phase_orderings = itertools.permutations([9,8,7,6,5])

max_out = -np.pi
for j, phase_setting_ordering in enumerate(phase_orderings):
    in_queues = []
    for i, phase in enumerate(phase_setting_ordering):
        if i == 0:
            in_queues.append([phase, 0])
        else:
            in_queues.append([phase])
    out_queues = in_queues[1:]+[in_queues[0]]

    amplifier_names = 'abcde'
    amps = []
    for name, in_queue, out_queue in zip(amplifier_names, in_queues, out_queues):
        amps.append(amplifier(name, program, in_queue, out_queue))

    continue_ = True
    while continue_:
        for amp in amps:
            amp.run_until_inputop_and_empty_input_queue_or_end_of_program()
            if amp.name == 'e' and amp.stopped is True:
                continue_ = False
    out = amps[-1].output_queue[0]
    if out > max_out:
        max_out = out
        print(j, max_out)
max_out

0 17485553
6 18013440
24 25932499
30 34908883
32 34909445
54 35960544
56 35961106


35961106